# Sprint 3 · Webinar 10 — Clase Práctica de SQL (SQL Workbench en línea)

**Enfoque:** practicar SQL en navegador usando DuckDB (sin Python) desde https://sql-workbench.com/

**Duración sugerida:** 100 min

---

## Dataset (CSV en GitHub RAW)

- customers: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_customers.csv
- orders: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_orders.csv
- order_items: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_order_items.csv
- payments: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_payments.csv
- products: https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_products.csv

> **Nota:** En `order_items` usaremos `unit_price` (no `price`).  
> En `products` usaremos `cost` para estimar margen.


## Objetivos de aprendizaje

Al final de esta sesión, podrás:

1. Cargar CSVs en DuckDB desde URLs (sin instalar nada).
2. Combinar tablas con `JOIN` usando llaves (`customer_id`, `order_id`, `product_id`).
3. Calcular KPIs básicos con `COUNT`, `SUM`, `AVG` y expresiones aritméticas.
4. Resumir resultados con `GROUP BY`, filtrar con `HAVING` y ordenar con `ORDER BY`.
5. Resolver ejercicios sin usar CTEs (`WITH ... AS`) — solo SQL “directo” y legible.


## Agenda (100 min)

- **(10 min)** Configuración en SQL Workbench + carga de dataset
- **(8 min)** Sanity checks (verificar tablas, conteos, columnas clave)
- **(10 min)** **Ejercicio 0** — Breakout Rooms (calentamiento)
- **(62 min)** Ejercicios guiados (1 a 5)
- **(5 min)** Retos (opcional) + cierre + takeaways


## 0) Configuración rápida en SQL Workbench (DuckDB)

1. Abre: https://sql-workbench.com/
2. Asegúrate de estar usando **DuckDB** (modo por defecto del sitio).
3. Copia y ejecuta **todo** este bloque para crear vistas (tablas virtuales) desde CSV:

```sql
-- 0) Limpieza (por si ya existían)
DROP VIEW IF EXISTS customers;
DROP VIEW IF EXISTS orders;
DROP VIEW IF EXISTS order_items;
DROP VIEW IF EXISTS payments;
DROP VIEW IF EXISTS products;

-- 1) Crear vistas desde CSV en GitHub (RAW)
CREATE OR REPLACE VIEW customers AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_customers.csv');

CREATE OR REPLACE VIEW orders AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_orders.csv');

CREATE OR REPLACE VIEW order_items AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_order_items.csv');

CREATE OR REPLACE VIEW payments AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_payments.csv');

CREATE OR REPLACE VIEW products AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_products.csv');
```

✅ Si no te carga: revisa que estés copiando la URL **raw** (no la de “vista” de GitHub) y que no haya espacios extra.


## 1) Verificación rápida (Sanity checks)

### 1.1 ¿Las vistas existen?

```sql
SHOW TABLES;
```

### 1.2 ¿Cuántos registros hay por tabla?

```sql
SELECT 'customers'   AS tabla, COUNT(*) AS n FROM customers
UNION ALL
SELECT 'orders'      AS tabla, COUNT(*) AS n FROM orders
UNION ALL
SELECT 'order_items' AS tabla, COUNT(*) AS n FROM order_items
UNION ALL
SELECT 'payments'    AS tabla, COUNT(*) AS n FROM payments
UNION ALL
SELECT 'products'    AS tabla, COUNT(*) AS n FROM products;
```

### 1.3 Mira 5 filas de cada tabla (para “oler” los datos)

```sql
SELECT * FROM customers LIMIT 5;
SELECT * FROM orders LIMIT 5;
SELECT * FROM order_items LIMIT 5;
SELECT * FROM payments LIMIT 5;
SELECT * FROM products LIMIT 5;
```

> Tip rápido: si una columna “no existe”, suele ser por un nombre distinto (`unit_price` vs `price`, etc.).


## 2) Ejercicio 0 (Breakout Rooms)

### Ejercicio 0 — Calentamiento (Breakout Rooms) (10 min)

**Dinámica (Zoom Breakout Rooms):**
- En grupos de 3–4, una persona comparte pantalla y otra dicta.
- Primero intentan 6–7 min sin ver la solución.
- Últimos 3–4 min: comparan resultados y anotan dudas para plenaria.

**Contexto:** queremos un mini-resumen del negocio para “romper el hielo”.

**Objetivo:** calcular, por **estado de la orden** y **método de pago**:
- número de órdenes
- total pagado (`SUM(amount_paid)`)

**Conceptos que combina (baja dificultad):**
- `JOIN` + `GROUP BY` + `COUNT(DISTINCT ...)` + `ORDER BY`

**Tablas:** `orders`, `payments`

**Pistas:**
1. Une `orders` con `payments` por `order_id`.
2. Agrupa por `orders.status` y `payments.payment_method`.
3. Usa `COUNT(DISTINCT o.order_id)` para contar órdenes.
4. Ordena por `total_pagado` descendente.

**Solución:**
```sql
SELECT
  o.status,
  p.payment_method,
  COUNT(DISTINCT o.order_id) AS n_orders,
  SUM(p.amount_paid)         AS total_pagado
FROM orders o
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY
  o.status,
  p.payment_method
ORDER BY total_pagado DESC;
```


## 3) Ejercicios guiados

> Reglas de la sesión:  
> - **No usar CTEs** (`WITH ... AS`).  
> - Si necesitas “pasos intermedios”, usa una tabla temporal (`CREATE TEMP TABLE`) o repite la subconsulta (preferimos legibilidad).

--- 


### Ejercicio 1 — Órdenes por segmento de cliente (12 min)

**Objetivo:** identificar qué tipo de clientes compra más.

**Qué debes obtener (por `segment`):**
- número de órdenes (`n_orders`)
- total pagado (`total_pagado`)
- ticket promedio (`avg_ticket`)

**Tablas:** `customers`, `orders`, `payments`

**Descripción:** vas a unir cliente → orden → pago y luego resumir por segmento.

**Pistas:**
- `customers` → `orders` por `customer_id`
- `orders` → `payments` por `order_id`
- `avg_ticket` = `SUM(amount_paid) / COUNT(DISTINCT order_id)`

**Solución:**
```sql
SELECT
  c.segment,
  COUNT(DISTINCT o.order_id) AS n_orders,
  SUM(p.amount_paid)         AS total_pagado,
  SUM(p.amount_paid) / COUNT(DISTINCT o.order_id) AS avg_ticket
FROM customers c
JOIN orders o
  ON c.customer_id = o.customer_id
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY c.segment
ORDER BY total_pagado DESC;
```


### Ejercicio 2 — Ingresos por categoría (GROUP BY + JOIN) (15 min)

**Objetivo:** calcular ingresos (revenue) por `category`.

**Descripción:** vamos a calcular revenue por línea de ítem y luego sumar por categoría.

**Cómo calcular revenue por ítem:**
- `revenue_item = quantity * unit_price * (1 - discount_rate)`

**Tablas:** `order_items`, `products`

**Pistas:**
- Une `order_items` con `products` por `product_id`.
- Agrupa por `products.category`.
- Usa `SUM(...)` para el revenue total.

**Solución:**
```sql
SELECT
  pr.category,
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) AS revenue
FROM order_items oi
JOIN products pr
  ON oi.product_id = pr.product_id
GROUP BY pr.category
ORDER BY revenue DESC;
```


### Ejercicio 3 — Top 5 productos por revenue (ORDER BY + LIMIT) (12 min)

**Objetivo:** encontrar los 5 productos que más dinero generan.

**Qué debes mostrar:**
- `product_id`, `title`, `category`
- revenue total del producto

**Tablas:** `order_items`, `products`

**Pistas:**
- Une `order_items` con `products`.
- Agrupa por `product_id`, `title`, `category`.
- `ORDER BY revenue DESC` y `LIMIT 5`.

**Solución:**
```sql
SELECT
  pr.product_id,
  pr.title,
  pr.category,
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) AS revenue
FROM order_items oi
JOIN products pr
  ON oi.product_id = pr.product_id
GROUP BY
  pr.product_id,
  pr.title,
  pr.category
ORDER BY revenue DESC
LIMIT 5;
```


### Ejercicio 4 — Margen bruto por categoría (20 min)

**Objetivo:** estimar rentabilidad por categoría.

**Descripción:** con revenue y costo estimamos ganancia bruta y margen.

**Cálculos:**
- `revenue = SUM(quantity * unit_price * (1 - discount_rate))`
- `cost = SUM(quantity * cost)`  *(cost viene de `products`)*
- `gross_profit = revenue - cost`
- `margin_pct = 100 * gross_profit / revenue`

**Tablas:** `order_items`, `products`

**Pistas:**
- Puedes repetir la expresión de revenue dentro del `CASE` para evitar división por cero.
- Usa alias y ordena por `margin_pct` o `gross_profit`.

**Solución (sin CTE):**
```sql
SELECT
  pr.category,

  -- Revenue
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) AS revenue,

  -- Cost
  SUM(oi.quantity * pr.cost) AS cost,

  -- Gross Profit
  SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate))
  - SUM(oi.quantity * pr.cost) AS gross_profit,

  -- Margin %
  CASE
    WHEN SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate)) = 0 THEN NULL
    ELSE
      100.0 *
      (
        SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate))
        - SUM(oi.quantity * pr.cost)
      )
      / SUM(oi.quantity * oi.unit_price * (1 - oi.discount_rate))
  END AS margin_pct

FROM order_items oi
JOIN products pr
  ON oi.product_id = pr.product_id
GROUP BY pr.category
ORDER BY margin_pct DESC;
```


### Ejercicio 5 — ¿Las promos sirven? (CASE + GROUP BY + HAVING) (18 min)

**Objetivo:** comparar órdenes **con promo** vs **sin promo**.

**Descripción:** clasificaremos cada orden en un grupo y calcularemos KPIs por grupo.

**Qué debes obtener por grupo:**
- `grupo_promo`: 'con_promo' / 'sin_promo'
- `n_orders`
- `total_descuento` (desde `payments.discount`)
- `total_pagado` (desde `payments.amount_paid`)
- `avg_ticket`

**Tablas:** `orders`, `payments`

**Pistas:**
- `promo_code` puede venir vacío o nulo (trátalo como “sin promo”).
- Usa `CASE WHEN ... THEN ... ELSE ... END`.
- Agrega `HAVING` si quieres quedarte con grupos con suficientes órdenes.

**Solución:**
```sql
SELECT
  CASE
    WHEN o.promo_code IS NULL OR o.promo_code = '' THEN 'sin_promo'
    ELSE 'con_promo'
  END AS grupo_promo,

  COUNT(DISTINCT o.order_id) AS n_orders,
  SUM(p.discount)            AS total_descuento,
  SUM(p.amount_paid)         AS total_pagado,
  SUM(p.amount_paid) / COUNT(DISTINCT o.order_id) AS avg_ticket

FROM orders o
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY 1
HAVING COUNT(DISTINCT o.order_id) >= 10
ORDER BY total_pagado DESC;
```

> Nota: Si tu dataset es pequeño o el filtro te deja vacío, baja el umbral del `HAVING` (por ejemplo, 5 o quítalo).


## 7) Retos (opcional, para casa)

### Reto A — Estados con mayor tasa de cancelación
**Objetivo:** identificar estados de orden (status) que “duelen” por cancelación.

**Pistas:**
- Solo necesitas `orders`.
- Calcula: `cancel_rate = 100 * canceladas / total`.

**Solución (una opción):**
```sql
SELECT
  status,
  COUNT(*) AS n_orders
FROM orders
GROUP BY status
ORDER BY n_orders DESC;
```

### Reto B — Clientes “top” por gasto total
**Objetivo:** Top 10 clientes por `amount_paid`.

**Pistas:**
- `customers` → `orders` → `payments`
- Agrupa por `customer_id` (y si quieres, nombre/email).

**Solución:**
```sql
SELECT
  c.customer_id,
  c.email,
  SUM(p.amount_paid) AS total_paid
FROM customers c
JOIN orders o
  ON c.customer_id = o.customer_id
JOIN payments p
  ON o.order_id = p.order_id
GROUP BY
  c.customer_id,
  c.email
ORDER BY total_paid DESC
LIMIT 10;
```


## Cierre (5 min)

**Takeaways**
- Si puedes leer bien el problema, puedes resolverlo con: `SELECT` + `JOIN` + `GROUP BY`.
- En datasets “reales”, el 80% del trabajo es:
  - unir tablas correctamente,
  - validar conteos,
  - y construir métricas con `SUM/COUNT/AVG`.

**Checklist de calidad**
- ¿Tus `JOIN` duplican filas? (usa `COUNT(DISTINCT ...)` para validar)
- ¿Tus fórmulas de revenue/margen tienen sentido?
- ¿Tus filtros (`WHERE/HAVING`) están en el lugar correcto?



## Siguientes pasos

- Repite los ejercicios 2–4 cambiando el agrupamiento:
  - por `category`, luego por `author` (si existe), luego por `payment_method`.
- En la siguiente sesión: construiremos un **reporte** más completo (tabla final) y lo exportaremos.

